In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%run functions/datetime_functions.py


np.set_printoptions(formatter={'float_kind':'{:f}'.format})
pd.set_option('display.float_format', lambda x: '%.3f' % x)
plt.style.use('seaborn')

## Loading the data

In [7]:
%%time
path = r"/home/dazai/Documents/Processmining/Data/"
df_train = pd.read_csv(f'{path}BPI Challenge 2017-test.csv')
df_test = pd.read_csv(f'{path}BPI Challenge 2017-training.csv')
df_data = pd.concat([df_train, df_test])

# Converting to datetime
remove_ms(df_data)
df_data['event time:timestamp'] = f_memoize_dt(df_data['event time:timestamp'])



df_data.sort_values(by=['event time:timestamp'], inplace=True)
df_data.reset_index(inplace=True, drop=True)

# remove whitespace at beginning and end of column name
df_data.columns = df_data.columns.str.strip(' ')

CPU times: user 8.48 s, sys: 284 ms, total: 8.76 s
Wall time: 8.76 s


In [13]:
for i in range(0,1202267, 10000):
    display(df_data['event time:timestamp'].iloc[[i]])

0   2016-01-01 10:51:15
Name: event time:timestamp, dtype: datetime64[ns]

10000   2016-01-09 14:55:21
Name: event time:timestamp, dtype: datetime64[ns]

20000   2016-01-14 11:15:37
Name: event time:timestamp, dtype: datetime64[ns]

30000   2016-01-19 09:17:26
Name: event time:timestamp, dtype: datetime64[ns]

40000   2016-01-22 13:32:46
Name: event time:timestamp, dtype: datetime64[ns]

...

1160000   2016-12-24 11:04:08
Name: event time:timestamp, dtype: datetime64[ns]

1170000   2016-12-28 20:21:21
Name: event time:timestamp, dtype: datetime64[ns]

1180000   2017-01-02 11:15:28
Name: event time:timestamp, dtype: datetime64[ns]

1190000   2017-01-06 15:17:56
Name: event time:timestamp, dtype: datetime64[ns]

1200000   2017-01-23 08:44:22
Name: event time:timestamp, dtype: datetime64[ns]